# Goal is to look into using codeBERT
* Want to  tokenize the entire training dataset

Takeaways: yeah that works

In [ ]:
import pandas as pd
import numpy as np
import pickle
from transformers import AutoTokenizer, AutoModel
from datasets import Dataset
from datasets import load_dataset
import datasets
import torch
from collections import Counter
import string
from scipy import spatial


In [ ]:
print(torch.backends.cudnn.enabled)
print(torch.cuda.is_available()) #We have GPU on deck and ready
print(f"CUDA device: {torch.cuda.get_device_name(torch.cuda.current_device())}")

In [ ]:
dataset_dict = datasets.load_from_disk("./Dataset/CodeSearchCorpus/")

In [ ]:
train_dataset = dataset_dict["train"]
train_dataset

In [ ]:
train_dataset = train_dataset.select_columns("whole_func_string")
# train_dataset_DF = train_dataset.to_pandas()

In [ ]:
train_dataset[0]

In [ ]:
model_ckpt = "huggingface/CodeBERTa-small-v1" #Can/Should test different models
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model = AutoModel.from_pretrained(model_ckpt)

In [ ]:
# Load the model to the GPU. Mine is a 3060
device = torch.device("cuda")
model.to(device)

In [ ]:
#From Hugging Face Tutorials
def cls_pooling(model_output):
    return model_output.last_hidden_state[:, 0]

def get_embeddings(text_list):
    encoded_input = tokenizer(
        text_list, padding=True, truncation=True, return_tensors="pt"
    )
    encoded_input = {k: v.to(device) for k, v in encoded_input.items()}
    model_output = model(**encoded_input)
    return cls_pooling(model_output)

In [ ]:
# num_rows = 50000
# filepath_pkl_obj = "./PickleObjects/"
# inverted_index_name = f"inverted_index_{num_rows}.pkl"
# tsed_name = f"train_subset_embeddings_dataset_{num_rows}.pkl"

# print(inverted_index_name, tsed_name)

In [ ]:
# np.random.seed(1)
# num_rows = 50000
# train_subset_indices = np.random.choice(len(train_dataset), num_rows, replace = False)
# train_dataset_subset = train_dataset.select(train_subset_indices)

# len(train_dataset_subset)

In [ ]:
# train_embeddings_dataset = train_dataset.map(
#         lambda x: {"embeddings": get_embeddings(x["whole_func_string"]).detach().cpu().numpy()[0]}
#     )

# train_embeddings_dataset.add_faiss_index(column="embeddings")

# with open(f'./pickleObjects/tsed_codeBERT_FULL.pkl', 'wb') as f:  # open a text file
#     pickle.dump(train_embeddings_dataset, f) # serialize the list
#     f.close()

In [ ]:
# train_embeddings_dataset
# with open(f'./pickleObjects/ted_codeBERT_FULL.pkl', 'wb') as f:  # open a text file
#     pickle.dump(train_embeddings_dataset, f) # serialize the list
#     f.close()

In [ ]:
train_subset_embeddings_dataset = train_dataset_subset.map(
        lambda x: {"embeddings": get_embeddings(x["whole_func_string"]).detach().cpu().numpy()[0]}
    )

train_subset_embeddings_dataset.add_faiss_index(column="embeddings")

with open(f'./pickleObjects/tsed_codeBERT_50K.pkl', 'wb') as f:  # open a text file
    pickle.dump(train_subset_embeddings_dataset, f) # serialize the list
    f.close()